In [ ]:
# return the position of the vertical lines of the table
def get_Ver_Lines(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    VerImg =np.zeros(image.shape).astype(np.uint8)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,25))
    vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    goodc=[]
    
    image_height=image.shape[0]
    min_height=int(.8*image_height)
    
    for c in cnts:
        rect = cv2.minAreaRect(c) 
        width = rect[1][0]
        height = rect[1][1]
        if ((width >= min_height) or (height >= min_height)):
                goodc.append(c)
    
    LinesPos = []
    # print(len(goodc))
    for c in goodc:
        cv2.drawContours(VerImg, [c], -1, (255,255,255), 2)
        LinesPos.append([ min(c[:,0][:,0]), max(c[:,0][:,0]) ])
    LinesPos.sort()

    
    # show_images([VerImg],["VerImg"])
    
    return LinesPos,


In [ ]:
# get the position of the Horizontal lines of the Table
def get_HOR_Lines(image,lineWidthPercenatge):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 41, 5)
    
    HorImg =np.zeros(image.shape).astype(np.uint8)

    # Find number of rows 
    image_width=image.shape[1]
    min_width=int(image_width*lineWidthPercenatge)

    horizontal_kernel_Small = cv2.getStructuringElement(cv2.MORPH_RECT , (min_width//2,1))
    thresh_lttleBigger = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel_Small, iterations=1)


    kernal_Diolate_before_hor = np.ones((10,min_width), np.uint8)
    VerticalDiolateBeforeHor = cv2.morphologyEx(thresh_lttleBigger, cv2.MORPH_CLOSE, kernal_Diolate_before_hor)


    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT , (min_width,1))
    horizontal = cv2.morphologyEx(VerticalDiolateBeforeHor, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
    
    
    kernal_Diolate = np.ones((3,int(image_width*1.6)), np.uint8)
    HorImgDia = cv2.morphologyEx(horizontal, cv2.MORPH_DILATE, kernal_Diolate)
    
    Thin_HOR = thin(HorImgDia, 10).astype(np.uint8)


    cnts = cv2.findContours(Thin_HOR, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    goodc=[]
    min_width_contours=int(.6*image_width)
    
    for c in cnts:
        rect = cv2.minAreaRect(c) 
        width = rect[1][0]
        height = rect[1][1]
        if (( (width >= min_width) or (height >= min_width) )):
                goodc.append(c)
                
    LinesPos=[]
    for c in goodc:
        cv2.drawContours(HorImg, [c], -1, (255,255,255), thickness=3)
        LinesPos.append(min(c[:,0][:,1]))
    LinesPos.sort()

    accepted = [] 
    accepted = [LinesPos[0]]
    avgCellHeight = getAvgDiff(LinesPos)
    for i in range(1,len(LinesPos)):
        if(LinesPos[i] - LinesPos[i-1] >= int(avgCellHeight*.5)):
            accepted.append(LinesPos[i])

    if(len(accepted) != len(LinesPos)):
        accepted[0] -= 15

    return accepted

In [ ]:
#exctract that exact colum from colum image 
def exestractCloum(colum):
    edged = getEdges(colum)
    kernel = np.ones((20,20), np.uint8)
    img_dilation = cv2.dilate(edged, kernel, iterations=1)
    docCnt = getContours(img_dilation)
    PureColum = colum
    if(docCnt is not None):
        PureColum = exetractTable(colum,docCnt)
    show_images([colum, edged , PureColum],["colum ", "edged" , "PureColum"])

In [ ]:
def dist(p1, p2):
    (x1, y1), (x2, y2) = p1, p2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

#get the avagrage diffrent between the vertical lines
def getAvgDiff(LinesPos):
    LinesPos.sort()
    avg_distance=0
    for i in range(1,len(LinesPos)):
        avg_distance += abs(LinesPos[i] - LinesPos[i-1])
    avg_distance = avg_distance / len(LinesPos)
    return avg_distance
